## 计算字符串编辑距离 edit distance

Smith-Waterman algorithm

https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm

Non-negative socre is suitable for local alignment.

$ H_{ij} = max \left \{
  \begin{aligned}
    H_{i-1,j-1} + S_{i,j} \\
    max_{k>=1} \{ H_{i-k,j} - W_k\} \\
    max_{l>=1} \{ H_{i,j-l} - W_l\} \\
    0
  \end{aligned} \right.$


In [ ]:
import numpy as np
np.set_printoptions(precision=2)

In [ ]:
def smith_waterman(list_1, list_2):
    def substitution(a,b):
        if a == b:
            return 3
        else: 
            return -3

    def penalty(k):
        return 2*k

    def score(h, i, j):
        match = h[i-1,j-1]+substitution(list_1[i-1], list_2[j-1])
        delete = max([h[i-k,j]-penalty(k) for k in range(0,i)])
        insert = max([h[i,j-l]-penalty(l) for l in range(0,j)])
        return max(match,delete,insert,0)
    
    # init
    n = len(list_1)
    m = len(list_2)
    H = np.zeros((n+1,m+1))
    H[:, 0] = 0
    H[0, :] = 0

    # fill score matrix   
    i = j = 1
    for i in range(1,n+1):
        for j in range(1,m+1):
            H[i,j] = score(H, i, j)
    print(H)
    
    # backtrace
    H_flip = np.flip(np.flip(H, 0), 1)
    start_i, start_j = np.unravel_index(np.argmax(H_flip, axis=None), H_flip.shape)
    start_i = n - start_i
    start_j = m - start_j
    
    def pair(i,j):
        return {'i': i,'j': j,'pair': list_1[i-1] + ' - ' + list_2[j-1]}
    
    path = []
    while H[start_i, start_j] > 0:
        path.append(pair(start_i,start_j))
        v1 = H[start_i-1, start_j]
        v2 = H[start_i, start_j-1]
        v3 = H[start_i-1, start_j-1]
        if v1 >= v2 and v1 >= v3:
            start_i = start_i - 1
        elif v2 >= v1 and v2 >= v3:
            start_j = start_j - 1
        elif v3 >= v1 and v3 >= v2:
            start_i = start_i - 1
            start_j = start_j - 1
        
    path.reverse()
    return path

In [ ]:
sw = smith_waterman(["G","G","T","T","G","A","C","T","A"], ["T","G","T","T","A","C","G","G","C"])
print(sw,'\n')

Needleman-Wunsch algorithm

$ H_{ij} = max \left \{
  \begin{aligned}
    H_{i-1,j-1} + S_{i,j} \\
    H_{i-k,j} - W \\
    H_{i-1,j} - W \\
  \end{aligned} \right.$


In [ ]:
def needleman_wunsch(list_1,list_2):

    def substitution(a,b):
        if a == b:
            return 1
        else: 
            return -1
    def penalty(k):
        return k

    # diff than smith_waterman
    def score(h, i, j):
        match = h[i-1,j-1]+substitution(list_1[i-1], list_2[j-1])
        delete = h[i,j-1]-penalty(1)
        insert = h[i-1,j]-penalty(1)
        return max(match,delete,insert)

    n = len(list_1)
    m = len(list_2)
    H = np.zeros((n+1,m+1),dtype=np.int32)
    H[:, 0] = range(0,-n-1,-1)
    H[0, :] = range(0,-m-1,-1)
    
    i = j = 1
    for i in range(1,n+1):
        for j in range(1,m+1):
            H[i,j] = score(H, i, j)
    print(H)

    # backtrace
    # might branch into diff paths
    start_i = n
    start_j = m
    def pair(i,j):
        return {'i': i,'j': j,'pair': list_1[i-1] + ' - ' + list_2[j-1]}
    
    path = []
    while start_i >= 0 or start_j >= 0:
        path.append(pair(start_i,start_j))
        v1 = H[start_i-1, start_j]
        v2 = H[start_i, start_j-1]
        v3 = H[start_i-1, start_j-1]
        if v1 >= v2 and v1 >= v3:
            start_i = start_i - 1
        elif v2 >= v1 and v2 >= v3:
            start_j = start_j - 1
        elif v3 >= v1 and v3 >= v2:
            start_i = start_i - 1
            start_j = start_j - 1
        
    path.reverse()
    return path

In [ ]:
nw = needleman_wunsch(["G","A","C","T","T","A","C"], ["C","G","T","G","A","A","T","T","C","A","T"])
print(nw,'\n')

N-gram algorithm

https://webdocs.cs.ualberta.ca/~kondrak/papers/spire05.pdf

three variants: binary, comprehensive, positional

In [ ]:
# unigram edit distance
# x, y are single characters
# X, Y are listsa
def d_single(x,y):
    if x != None and y == None:
        return 1
    if x == None and y != None:
        return 1
    if x == y:
        return 0
    else:
        return 1
def d1(X,Y):    
    k = len(X)
    l = len(Y)
    gamma = np.zeros((k+1,l+1))
    gamma[:,0] = [i for i in range(k+1)]
    gamma[0,:] = [j for j in range(l+1)]
    for i in range(1,k+1):
        for j in range(1,l+1):
            gamma[i,j] = min(
                gamma[i-1,j]+1,
                gamma[i,j-1]+1,
                gamma[i-1,j-1] + d_single(X[i-1],Y[j-1])
            )
    return gamma[k,l]

# n-gram edit distance

# 3 variants
def positional(list_x, list_y):
    n = len(list_x)
    s = [d_single(list_x[i],list_y[i]) for i in range(n)]
    return sum(s) / n

def comprehensive(X, Y):
    n = len(X)
    return d1(X,Y) / n

def binary(X,Y):
    for i in range(len(X)):
        if X[i] != Y[i]:
            return 1
    return 0
 
def dn(X, Y, n, variant):
    k = len(X)
    l = len(Y)
    X = [None]*(n-1) + X
    Y = [None]*(n-1) + Y
    D = np.empty((k+1,l+1))
    D[:,0] = [i for i in range(k+1)]
    D[0,:] = [j for j in range(l+1)]
    for i in range(1,k+1):
        for j in range(1,l+1):
            D[i,j] = min(
                D[i-1,j]+1,
                D[i,j-1]+1,
                D[i-1,j-1] + variant(X[i-1:i+n-1],Y[j-1:j+n-1])
            )            
    print(D)
    return D[k,l] / max(k,l)

In [ ]:
l1 = [l for l in "qbcdefarjn"]
l2 = [l for l in "abcdeaweqg"]
n = 4
print(l1,'\n')
print(l2,'\n')

bdn = dn(l1,l2,n,binary)
print('binary \n' ,bdn)

cdn = dn(l1,l2,n,comprehensive)
print('comprehensive \n', cdn)

pdn = dn(l1,l2,n,positional)
print('positional \n', pdn)

# HAS BUG
# cdn is always equal to pdn